# 利用直接消元法以及广义消元法求解等式约束的二次规划问题

\begin{equation*}
\begin{array}{cl}
\text{minimize} & q(x) = x_1^2 + x_2^2 + 2x_3^2 + 5x_1x_2 - 3x_2 - 7x_3, \\
\text{subject to} & x_1 + x_2 + x_3 = 1, \\
& x_1 - 2x_2 - 3x_3 = -2,
\end{array}
\end{equation*}


In [ ]:
import sympy as sp

In [ ]:
x1, x2, x3 = sp.symbols("x1, x2, x3")
qx = x1**2 + x2**2 + 2 * x3**2 + 5 * x1 * x2 - 3 * x2 - 7 * x3

In [ ]:
G = sp.Matrix(3, 3, [2, 5, 0, 5, 2, 0, 0, 0, 4])
d = sp.Matrix(3, 1, [0, -3, -7])
A = sp.Matrix(3, 2, [1, 1, 1, -2, 1, -3])
b = sp.Matrix(2, 1, [1, -2])

## 直接消元法

In [ ]:
psi = sp.simplify(qx.subs([(x1, sp.Rational(1, 3) * x3), (x2, 1 - sp.Rational(4, 3) * x3)]))
psi

In [ ]:
sp.diff(psi, (x3, 2))

In [ ]:
sp.diff(psi, (x3, 1))

In [ ]:
g = - G @ sp.Matrix(3, 1, [sp.Rational(2, 5), -sp.Rational(3, 5), sp.Rational(6, 5)]) - d
g

In [ ]:
sp.linsolve((A, g))

## 广义消元法

In [ ]:
q, r = A.QRdecomposition()

In [ ]:
q

In [ ]:
r

In [ ]:
Y = q @ r.T.inv()
Y

In [ ]:
sp.linsolve((q.T, sp.zeros(2, 1))) == sp.linsolve((A.T, sp.zeros(2, 1)))

In [ ]:
sp.linsolve((q.T, sp.zeros(2, 1)))

In [ ]:
Z = sp.Matrix(3, 1, [1, -4, 3])
Z = Z / Z.norm()
Z

In [ ]:
Z.T @ G @ Z

In [ ]:
-Z.T @ (d + G @ Y @ b)

In [ ]:
y_star = (Z.T @ G @ Z).inv() @ (-Z.T @ (d + G @ Y @ b))
y_star

In [ ]:
x_star = Y @ b + Z @ y_star
x_star

In [ ]:
lambda_star = - Y.T @ (G @ x_star + d)
lambda_star